In [6]:
# import modules
import face_recognition as fr
import numpy as np
import cv2
import os

In [7]:
class ObjectD_FaceR:
    
    ### GLOBAL VARIABLES ###
    
    # object detection variables
    net = None # network variable
    object_names = [] # object names storage
    colors = None # color storage
    
    # face recognition variables
    images = []  # images storage
    classNames = [] # names storage
    knownEncodeList = [] # known encodes storage
    
    path = 'images_path/train_images_2' # encoded path for the location of images that will be trained
    myList = os.listdir(path) # accessing all files in the directed path
    
    savedImage = [] # storage of latest saved image
    
    ### GLOBAL VARIABLES ###
    
    # default constructor
    def __init__(self):
        
        # create network model using yolov3
        self.net = cv2.dnn.readNet('yolov3/yolov3.weights', 'yolov3/yolov3-416.cfg')

        # extract object_names from coco.names file
        with open('yolov3/coco.names','r') as f:
            self.object_names = f.read().splitlines()
            
        # create random colors (parameters: colors = between 0 and 255 colors, size = number of random colors and channels)    
        self.colors = np.random.uniform(0,255, size = (len(self.object_names), 3)) 
        
        # call preTrainedFaces function to generate known face encodings
        self.knownEncodeList = self.preTrainedFaces(self.myList)
        
        
    def getEncodings(self,images):
        
        encodeList = [] # encodes temp storage
        
        # generate encodings on each images
        for img in images:

            # convert image to RGB, (face recognition only accepts RGB format)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # generate and append encodings
            encode = fr.face_encodings(img_rgb)[0]
            encodeList.append(encode)
            
        print("\nEncoding complete\n")
        # return encodings
        return encodeList
    
    
    def preTrainedFaces(self,myList):

        # extracting images and image names on the given path 
        for cl in myList:

            # extract and append images
            curImage = cv2.imread(f'{self.path}/{cl}')
            self.images.append(curImage)

            # extract names without file extension
            self.classNames.append(os.path.splitext(cl)[0].split('#')[0])

        # call encoding function to encode/train all default images
        return self.getEncodings(self.images)
    
    # writing image
    def writeImage(self, images):
        
        if len(images)==0:
            print("No Frame Detected")
        else:
            cv2.imwrite("Detections.jpg",self.savedImage)
            print("Image Saved")
    
    def ObjD_FaceR_Image(self, path):
        
        img = cv2.imread(path) 
        
        # obtain height and width
        height, width, _ = img.shape
        
        # create font (for later use)
        font = cv2.FONT_HERSHEY_PLAIN

        # create blob using blobFromImage function in order to feed the data into the neural network(net)
        blob = cv2.dnn.blobFromImage(img, 1/255, (318, 318), (0,0,0), swapRB = True, crop = False)
        
        # set the Input from the blob into the network
        self.net.setInput(blob)

        # get the output layers names
        output_layers_names = self.net.getUnconnectedOutLayersNames()

        # pass the output layers name into the net.forward to get the outputs
        layerOutputs = self.net.forward(output_layers_names)


        # visualizing results
        boxes = []
        confidences = []
        object_names_ids = []

        for output in layerOutputs: # extract the information from the layerOutputs
            for detection in output: # extract the information from each of the outputs

                # note: the first four elements is the locations of the bounding boxes and the fifth element is the confidence score/probability
                # store score predictions. score predictions starts at 6th element or index 5
                scores = detection[5:]
                # extract highest scores using np.argmax
                object_names_id = np.argmax(scores)
                # extract real scores/probality using the extracted highest scores
                confidence = scores[object_names_id]
                # check confidence if larger than 0.3 start to locate boxes
                if confidence > 0.5:
                    center_x = int(detection[0]*width)
                    center_y = int(detection[1]*height)
                    w = int(detection[2]*width)
                    h = int(detection[3]*height)

                    # get the positions of up and left corner
                    x = int(center_x - w/2)
                    y = int(center_y - h/2)

                    # append all obtained informations
                    boxes.append([x, y, w, h])
                    confidences.append((float(confidence)))            
                    object_names_ids.append(object_names_id)

        # NMSBoxes: parameters: boxes, obtained confidences, acceptable confidence level, default = 0.4
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
        
        # check if indexes is Not Null/ none detected, just to prevent error and continue run through the video file
        if len(indexes)!=0:
            
            # keeping track of the number of persons and objects detected per frame
            person_count = 0
            object_count = 0
            
            for i in indexes.flatten():
                
                if self.object_names[object_names_ids[i]] == 'person':
                    person_count+=1
                else:
                    object_count+=1
                    # extract x and y coordinates and width height of boxes from boxes
                    x, y, w, h = boxes[i]
                    # extract label from the object names
                    label = str(self.object_names[object_names_ids[i]]).upper()
                    # extract corresponding confidence
                    confidence = str(round(confidences[i],2))
                    # assign color for boxes
                    color = self.colors[object_names_ids[i]]
                    # create a rectange: parameters: img, coordinates, size, color, thickness
                    cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
                    cv2.rectangle(img, (x,y), (x+w, y+20), color, cv2.FILLED)

                    # put text: parameters: img, text, location, font, font-size, color, text-thickness
                    cv2.putText(img, label + " " + confidence, (x, y+20), font, 1, (255,255,255), 2)
            
            if person_count>0:
                print("Person Detected: "+ str(person_count))
                print("Object Detected: "+ str(object_count))
                image = self.faceRecognition(img)
                
                
        # auto scale large images for screen resolution
        if height > 700:
            # divide 700 by height to select adjustment value
            adjustment = height/700
            # resize the image
            image = cv2.resize(image, (round(width/adjustment),round(height/adjustment)))

            # save image for writing
            self.savedImage = image
            # show image/video/cam
            cv2.imshow("Image",image)
        else:
            # save image for writing
            self.savedImage = image
            # show image/video/cam
            cv2.imshow("Image",image)

        # wait until key 0/close is pressed    
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        return self.savedImage
    
    def ObjD_FaceR_Video(self, path):
        
        cap = None
        frame = 0
            
        if str(path) == '0':
            cap = cv2.VideoCapture(0)
        else:
            cap = cv2.VideoCapture(path)
        
        # will be use for saving frames later
        frame_width = int(cap.get(3)) 
        frame_height = int(cap.get(4)) 
        size = (frame_width, frame_height)
        result = cv2.VideoWriter('Detections.avi',  
                         cv2.VideoWriter_fourcc(*'MJPG'), 
                         20, size)
        
        while True:
            
            # capture frames in video file
            _, img = cap.read()
            
            # check if null value
            if np.shape(img) != ():
                # frame counting
                frame+=1
                print("\nFrame: "+ str(frame))

                # create font (for later use)
                font = cv2.FONT_HERSHEY_PLAIN

                # obtain height and width
                height, width, _ = img.shape

                # create blob using blobFromImage function in order to feed the data into the neural network(net)
                blob = cv2.dnn.blobFromImage(img, 1/255, (318, 318), (0,0,0), swapRB = True, crop = False)

                # set the Input from the blob into the network
                self.net.setInput(blob)

                # get the output layers names
                output_layers_names = self.net.getUnconnectedOutLayersNames()

                # pass the output layers name into the net.forward to get the outputs
                layerOutputs = self.net.forward(output_layers_names)


                # visualizing results
                boxes = []
                confidences = []
                object_names_ids = []

                for output in layerOutputs: # extract the information from the layerOutputs
                    for detection in output: # extract the information from each of the outputs

                        # note: the first four elements is the locations of the bounding boxes and the fifth element is the confidence score/probability
                        # store score predictions. score predictions starts at 6th element or index 5
                        scores = detection[5:]
                        # extract highest scores using np.argmax
                        object_names_id = np.argmax(scores)
                        # extract real scores/probality using the extracted highest scores
                        confidence = scores[object_names_id]
                        # check confidence if larger than 0.3 start to locate boxes
                        if confidence > 0.5:
                            center_x = int(detection[0]*width)
                            center_y = int(detection[1]*height)
                            w = int(detection[2]*width)
                            h = int(detection[3]*height)

                            # get the positions of up and left corner
                            x = int(center_x - w/2)
                            y = int(center_y - h/2)

                            # append all obtained informations
                            boxes.append([x, y, w, h])
                            confidences.append((float(confidence)))            
                            object_names_ids.append(object_names_id)

                # NMSBoxes: parameters: boxes, obtained confidences, acceptable confidence level, default = 0.4
                indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

                # check if indexes is Not Null/ none detected, just to prevent error and continue run through the video file
                if len(indexes)!=0:

                    # keeping track of the number of persons and objects detected per frame
                    person_count = 0
                    object_count = 0

                    for i in indexes.flatten():

                        if self.object_names[object_names_ids[i]] == 'person':
                            person_count+=1
                        else:
                            object_count+=1
                            # extract x and y coordinates and width height of boxes from boxes
                            x, y, w, h = boxes[i]
                            # extract label from the object names
                            label = str(self.object_names[object_names_ids[i]]).upper()
                            # extract corresponding confidence
                            confidence = str(round(confidences[i],2))
                            # assign color for boxes
                            color = self.colors[object_names_ids[i]]
                            # create a rectange: parameters: img, coordinates, size, color, thickness
                            cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
                            cv2.rectangle(img, (x,y), (x+w, y+20), color, cv2.FILLED)

                            # put text: parameters: img, text, location, font, font-size, color, text-thickness
                            cv2.putText(img, label + " " + confidence, (x, y+20), font, 1, (255,255,255), 2)

                    if person_count>0:
                        print("Person Detected: "+ str(person_count))
                        print("Object Detected: "+ str(object_count))
                        img = self.faceRecognition(img)

                # saving image ###### NOTE: comment out the result.write if you dont want to save the video frames #########
                result.write(img)
                # show image/video/cam
                cv2.imshow("Image",img)

                # break using esc key
                key = cv2.waitKey(1)
                if key==27:
                    break
            else:
                break
        # release captures     
        cap.release()
        result.release()
        
        # destroy all open windows
        cv2.destroyAllWindows()
        
    def faceRecognition(self,images):
        
        # resize image to increase running time, producing .25 -> 1/4 scale
        imgResize = cv2.resize(images, (0,0), None, 0.25,0.25)
        
        # convert image to RGB
        imgRGB = cv2.cvtColor(imgResize, cv2.COLOR_BGR2RGB)
        
        # extract face locations on each frame and generate encodes of the current faces in the frame
        # it will be use for comparing new faces with the trained faces 
        facesCurFrame = fr.face_locations(imgRGB)
        encodesCurFrame = fr.face_encodings(imgRGB,facesCurFrame)

        # comparing faces through encodings while keeping track of face locations
        for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):

            # check the face distance (the lower the value means the more identical the face is)
            faceDistance = fr.face_distance(self.knownEncodeList, encodeFace)
 
            # view face distance values
            print("Face Distances: "+str(faceDistance))         
            
            # extract minimum distance
            minDist = min(faceDistance)
            
            # check if face distance value is around the acceptable threshold(<0.5)-> this is only a made-up threshold
            if(minDist<0.5):
                
                # extract index of minimum distance
                index = np.argmin(faceDistance)
                
                # x1,y1 -> locations, x2,y2 -> width & height
                y1, x2, y2, x1 = faceLoc
    
                # rescale values, since we resize the image earlier
                y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4

                # extract face name
                name = self.classNames[index].upper()

                # draw rectangle in face
                cv2.rectangle(images, (x1,y1), (x2,y2), (255,0,0), 2)

                # draw filled rectangle at the top (emphasize text through background)
                cv2.rectangle(images, (x1,y1), (x2,y1+25), (255,0,0), cv2.FILLED)

                # put text
                cv2.putText(images, name, (x1, y1+20), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 2)
            else:
                
                # x1,y1 -> locations, x2,y2 -> width & height
                y1, x2, y2, x1 = faceLoc
    
                # rescale values, since we resize the image earlier
                y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4

                # unknown name
                name = 'UNKNOWN'

                # draw rectangle in face
                cv2.rectangle(images, (x1,y1), (x2,y2), (255,0,0), 2)

                # draw filled rectangle at the top (emphasize text through background)
                cv2.rectangle(images, (x1,y1), (x2,y1+25), (255,0,0), cv2.FILLED)

                # put text
                cv2.putText(images, name, (x1, y1+20), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 2)
            
        return images

In [8]:
# instantiate object -> automatically create image encodings on all files in the given path (all files must be image type)
detection = ObjectD_FaceR()


Encoding complete



In [10]:
# Video Input or Cam
# Pass integer value ''0'' to use Cam
detection.ObjD_FaceR_Video('images_path/test_videos/start_up.mp4')


Frame: 1
Person Detected: 1
Object Detected: 0
Face Distances: [0.41073882 0.42428735 0.37257766 0.39784645 0.52723135 0.47452403
 0.54745671 0.46649699 0.49105034 0.50047864 0.89119064 0.50355376
 0.50493621 0.43106345 0.44893096 0.49887088]

Frame: 2
Person Detected: 1
Object Detected: 0
Face Distances: [0.43443251 0.44321307 0.3863413  0.41836846 0.52363156 0.48454852
 0.54308256 0.45869612 0.4612289  0.46711302 0.86152626 0.47967066
 0.49940197 0.41657728 0.44116448 0.48464697]

Frame: 3
Person Detected: 1
Object Detected: 0
Face Distances: [0.43732598 0.44550641 0.38650719 0.41373706 0.5097674  0.46734144
 0.534739   0.45284681 0.4498844  0.45599462 0.8545858  0.46719373
 0.48699763 0.40793996 0.43002606 0.47177457]

Frame: 4
Person Detected: 1
Object Detected: 0
Face Distances: [0.42446766 0.4450788  0.38456808 0.40850743 0.51479428 0.46386474
 0.54117478 0.46163439 0.45964626 0.46623695 0.86631873 0.47778112
 0.49039586 0.4211727  0.44107559 0.48532737]

Frame: 5
Person Detecte

Person Detected: 1
Object Detected: 0
Face Distances: [0.42490145 0.47104154 0.39667638 0.43384246 0.54489949 0.49751079
 0.56787477 0.48394583 0.49082149 0.50737014 0.8750783  0.50788643
 0.47917642 0.41028375 0.43394344 0.49255277]

Frame: 36
Person Detected: 1
Object Detected: 0
Face Distances: [0.42320102 0.46807256 0.39741861 0.44546663 0.53870093 0.4921498
 0.55887282 0.47356424 0.50363007 0.5195098  0.88345647 0.52302971
 0.48296298 0.40730654 0.42426089 0.49187813]

Frame: 37
Person Detected: 1
Object Detected: 0
Face Distances: [0.42734301 0.46792643 0.39773721 0.4463805  0.5396489  0.49131407
 0.55732048 0.47148763 0.49437451 0.51167948 0.88109663 0.51491367
 0.48436268 0.4047559  0.4226232  0.48940167]

Frame: 38
Person Detected: 1
Object Detected: 0
Face Distances: [0.42102578 0.46527137 0.39609968 0.43959845 0.53674109 0.49250143
 0.55765709 0.46296454 0.49685271 0.51634995 0.8838243  0.5319557
 0.49031585 0.42023239 0.43976682 0.49359288]

Frame: 39
Person Detected: 1
Obj

Person Detected: 2
Object Detected: 0
Face Distances: [0.63786668 0.62072453 0.57342974 0.59465615 0.5425689  0.60174194
 0.61276114 0.52071742 0.31146569 0.3802189  0.83023776 0.36160124
 0.48915601 0.46649872 0.47455791 0.49143355]

Frame: 70
Person Detected: 2
Object Detected: 0
Face Distances: [0.61887544 0.60928525 0.56162985 0.57968516 0.53833652 0.59801405
 0.61061175 0.52004604 0.32250064 0.38878074 0.83340479 0.3635893
 0.47588629 0.46273687 0.46646626 0.50018369]

Frame: 71
Person Detected: 2
Object Detected: 0
Face Distances: [0.59373279 0.59906325 0.54589126 0.56958755 0.48337118 0.56074927
 0.55929422 0.47372692 0.30918463 0.3660182  0.82755223 0.3638469
 0.44889159 0.44856185 0.46494388 0.480784  ]

Frame: 72
Person Detected: 2
Object Detected: 0
Face Distances: [0.59575657 0.58952505 0.54076439 0.57178355 0.49772458 0.56159749
 0.57276022 0.48310612 0.32181529 0.37925269 0.83749347 0.35620722
 0.43850647 0.44862706 0.45554599 0.47273194]

Frame: 73
Person Detected: 2
Obj

Person Detected: 1
Object Detected: 0
Face Distances: [0.43109219 0.45257114 0.37870279 0.42864473 0.50863838 0.47241943
 0.53267606 0.42652408 0.4408219  0.46058929 0.84687562 0.46974655
 0.45848829 0.40988393 0.43905886 0.4678227 ]

Frame: 104
Person Detected: 1
Object Detected: 0
Face Distances: [0.41458561 0.43133244 0.35708412 0.41190802 0.52078331 0.48530257
 0.53569503 0.43361714 0.46129943 0.47944175 0.85687917 0.48083583
 0.46657679 0.40391965 0.45097299 0.47659223]

Frame: 105
Person Detected: 1
Object Detected: 0
Face Distances: [0.42141881 0.44232094 0.37115836 0.414672   0.51494284 0.48027405
 0.53429119 0.43458829 0.46728461 0.48712359 0.86078425 0.47839637
 0.46171759 0.4151605  0.45387495 0.479916  ]

Frame: 106
Person Detected: 1
Object Detected: 0
Face Distances: [0.41541437 0.44141753 0.369178   0.41062774 0.51121331 0.48976141
 0.53313553 0.43857036 0.43802857 0.46536622 0.84731457 0.47128075
 0.45105776 0.39998788 0.43395355 0.45495555]

Frame: 107
Person Detected:

Person Detected: 1
Object Detected: 0
Face Distances: [0.57511877 0.53848663 0.49353254 0.57161766 0.5446217  0.53177787
 0.54395407 0.46139609 0.4847452  0.47057834 0.83321314 0.53057366
 0.40558279 0.3341226  0.36322492 0.41557685]

Frame: 138
Person Detected: 1
Object Detected: 0
Face Distances: [0.59669127 0.54582935 0.51171391 0.57380306 0.58292032 0.5737045
 0.58947228 0.50807059 0.49241001 0.49492462 0.83840427 0.53058383
 0.48007262 0.38864105 0.42350378 0.46761922]

Frame: 139
Person Detected: 1
Object Detected: 0
Face Distances: [0.60518997 0.55824322 0.51997204 0.5950291  0.57949788 0.54436193
 0.58858277 0.51871462 0.52146099 0.5320205  0.83783025 0.56027149
 0.44693157 0.39403077 0.39311982 0.42286012]

Frame: 140
Person Detected: 1
Object Detected: 0
Face Distances: [0.6379482  0.59978834 0.54973881 0.62688271 0.59732074 0.55548134
 0.60321847 0.53445036 0.55122109 0.5669459  0.84107377 0.59813142
 0.49812769 0.44337324 0.42057985 0.47622596]

Frame: 141
Person Detected: 

...

Person Detected: 2
Object Detected: 0

Frame: 687
Person Detected: 2
Object Detected: 0

Frame: 688
Person Detected: 3
Object Detected: 0

Frame: 689
Person Detected: 3
Object Detected: 0

Frame: 690
Person Detected: 3
Object Detected: 0

Frame: 691
Person Detected: 1
Object Detected: 0

Frame: 692
Person Detected: 1
Object Detected: 0

Frame: 693
Person Detected: 3
Object Detected: 0

Frame: 694
Person Detected: 3
Object Detected: 0

Frame: 695
Person Detected: 2
Object Detected: 0

Frame: 696
Person Detected: 2
Object Detected: 0

Frame: 697
Person Detected: 2
Object Detected: 0

Frame: 698
Person Detected: 2
Object Detected: 0

Frame: 699
Person Detected: 3
Object Detected: 0

Frame: 700
Person Detected: 3
Object Detected: 0

Frame: 701
Person Detected: 2
Object Detected: 0

Frame: 702
Person Detected: 2
Object Detected: 0

Frame: 703
Person Detected: 1
Object Detected: 0

Frame: 704
Person Detected: 1
Object Detected: 0

Frame: 705
Person Detected: 1
Object Detected: 0

Frame: 706


Person Detected: 1
Object Detected: 0
Face Distances: [0.42115969 0.51418526 0.43372138 0.45909486 0.56574973 0.5748914
 0.58788886 0.52173671 0.54728626 0.55642082 0.80468846 0.54082076
 0.53814506 0.47830465 0.53964538 0.5593765 ]

Frame: 810
Person Detected: 1
Object Detected: 0
Face Distances: [0.40091342 0.49487855 0.41630609 0.44522214 0.5621617  0.56526948
 0.58457488 0.51039376 0.54563377 0.56284281 0.82105399 0.55441586
 0.54456739 0.4751082  0.51257258 0.55160366]

Frame: 811
Person Detected: 1
Object Detected: 0
Face Distances: [0.39477666 0.49908601 0.4243649  0.44483635 0.56263186 0.55683618
 0.58279978 0.51094774 0.54939051 0.56660286 0.84057833 0.55765665
 0.53469442 0.47607221 0.5201357  0.56591462]

Frame: 812
Person Detected: 1
Object Detected: 0
Face Distances: [0.38313527 0.4802591  0.40310743 0.42104614 0.54927482 0.53459375
 0.57046011 0.49311246 0.53493821 0.54945853 0.84793366 0.53945424
 0.51797607 0.46070917 0.50937403 0.55579725]

Frame: 813
Person Detected: 

Person Detected: 2
Object Detected: 0
Face Distances: [0.59261607 0.54751334 0.498752   0.53616902 0.5223148  0.49568663
 0.55612635 0.49749406 0.32968237 0.33390967 0.88198646 0.35336908
 0.48655504 0.41027534 0.4154413  0.4470857 ]

Frame: 863
Person Detected: 2
Object Detected: 0
Face Distances: [0.56725464 0.52228081 0.46485694 0.50831546 0.50141806 0.51111149
 0.54456602 0.48533312 0.27862488 0.28365979 0.84023039 0.34688418
 0.48407427 0.40382788 0.44776524 0.44489211]

Frame: 864
Person Detected: 2
Object Detected: 0
Face Distances: [0.61727482 0.55877662 0.5137825  0.54309512 0.56311743 0.56260712
 0.59334193 0.53959267 0.32213582 0.33607405 0.88683549 0.36197617
 0.52053079 0.41419889 0.4404346  0.44886527]

Frame: 865
Person Detected: 2
Object Detected: 0
Face Distances: [0.62135403 0.56836017 0.52076199 0.56242606 0.5594371  0.54908243
 0.58384447 0.52934351 0.33021689 0.3466912  0.88525945 0.37668969
 0.52812029 0.40050945 0.42246872 0.44037559]

Frame: 866
Person Detected:

Person Detected: 1
Object Detected: 0
Face Distances: [0.55714386 0.51906712 0.46761683 0.49578634 0.51441908 0.49439652
 0.54485872 0.46427885 0.36571517 0.36182644 0.87230953 0.39335078
 0.51300211 0.44361863 0.45113075 0.50743648]

Frame: 897
Person Detected: 1
Object Detected: 0
Face Distances: [0.54330917 0.50066008 0.45455884 0.48586581 0.47390195 0.44778602
 0.49728362 0.43624551 0.3633598  0.36232348 0.88189209 0.39556065
 0.51668314 0.43362964 0.44320295 0.50039084]

Frame: 898
Person Detected: 1
Object Detected: 0
Face Distances: [0.55689006 0.49627026 0.47529656 0.51493463 0.4739679  0.45383088
 0.49731006 0.44088811 0.42845841 0.41737133 0.88511033 0.45003788
 0.54317454 0.45221421 0.45594592 0.54512015]

Frame: 899
Person Detected: 1
Object Detected: 0
Face Distances: [0.52340534 0.48779299 0.4583757  0.47770838 0.44616367 0.4159902
 0.468262   0.39347847 0.38870801 0.39782376 0.88047979 0.43381678
 0.51105465 0.44373443 0.42981243 0.51195821]

Frame: 900
Person Detected: 

Person Detected: 4
Object Detected: 0

Frame: 991
Person Detected: 3
Object Detected: 0

Frame: 992
Person Detected: 3
Object Detected: 0

Frame: 993
Person Detected: 3
Object Detected: 0

Frame: 994
Person Detected: 3
Object Detected: 0

Frame: 995
Person Detected: 3
Object Detected: 0

Frame: 996
Person Detected: 2
Object Detected: 0

Frame: 997
Person Detected: 2
Object Detected: 0

Frame: 998
Person Detected: 2
Object Detected: 0

Frame: 999
Person Detected: 1
Object Detected: 0

Frame: 1000
Person Detected: 1
Object Detected: 0

Frame: 1001
Person Detected: 1
Object Detected: 0
Face Distances: [0.59516039 0.56692553 0.53774687 0.53729466 0.53118337 0.52048394
 0.56437038 0.4973498  0.37671923 0.44585877 0.84597655 0.45431111
 0.48256786 0.44134038 0.45029579 0.52061461]

Frame: 1002
Person Detected: 1
Object Detected: 0

Frame: 1003
Person Detected: 1
Object Detected: 0

Frame: 1004
Person Detected: 1
Object Detected: 0

Frame: 1005
Person Detected: 1
Object Detected: 0

Frame: 10

In [26]:
# Image Input
data = detection.ObjD_FaceR_Image('images_path/test_images/suzy.jpg')
# detection.ObjD_FaceR_Image('images_path/test_images/start_up.jpg')

detection.writeImage(data)

Person Detected: 3
Object Detected: 0
Face Distances: [0.34040125 0.43024443 0.40126013 0.41840024 0.48256617 0.44826615
 0.51057129 0.46115448 0.56698463 0.55895389 0.93760038 0.58811636
 0.50887042 0.44664741 0.52275606 0.57076094]
Image Saved
